# 기출 9회_작업형3

## 문제1. 다중 선형 회귀
주어진 데이터에서 id를 기준으로 다음과 같이 데이터를 분할하여 아래 소문제를 해결한다.
- train 데이터: 1 ≤ id ≤ 140
- test 데이터: id > 140

### 문제1-1
train 데이터에서 c1, c2, c3, c4, c5를 독립변수로, design을 종속변수로 하는 다중 회귀모형을 적합한 후, 절편을 제외한 독립변수 중 p-value가 0.05보다 작은 변수의 개수를 구하시오.

In [184]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_3/design.csv")
df.head()

,id,design,c1,c2,c3,c4,c5
0,1,58.962471,0.374540,0.642032,0.103124,0.168935,0.707239
1,2,42.006534,0.950714,0.084140,0.902553,0.278590,0.152539
2,3,55.831980,0.731994,0.161629,0.505252,0.177010,0.576288
3,4,64.458592,0.598658,0.898554,0.826457,0.088703,0.606715
4,5,61.342792,0.156019,0.606429,0.320050,0.120636,0.424131


In [185]:
# 1) train, test 데이터 분리
cond1 = df['id'] <= 140
cond2 = df['id'] > 140
train = df[cond1].copy()
test = df[cond2].copy()
# print(train.shape, test.shape) # (140, 7) (60, 7)

# 2) 전체 변수 사용 회귀분석
from statsmodels.formula.api import ols
model = ols("design ~ c1+c2+c3+c4+c5", data=train).fit()
print(model.summary())

# 3) 상수항(Intercept)은 제외하고, p-value가 0.05보다 작은 독립변수의 개수를 계산
print(sum(model.pvalues[1:]<0.05))

                            OLS Regression Results                            
Dep. Variable:                 design   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     9.697
Date:                Fri, 20 Jun 2025   Prob (F-statistic):           6.37e-08
Time:                        22:28:22   Log-Likelihood:                -468.72
No. Observations:                 140   AIC:                             949.4
Df Residuals:                     134   BIC:                             967.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     53.0220      2.294     23.112      0.0

### 문제1-2
문제 1-1에서 적합한 회귀모형의 결과를 바탕으로, p-value가 0.05 이하인 유의한 독립변수만을 선택하여 새로운 다중 회귀모형을 적합한 후, train 데이터에서 design의 예측값을 산출하고 예측값과 실제 design 값 사이의 피어슨 상관계수를 반올림하여 소수 셋째 자리까지 계산하시오.

In [195]:
# 방법1

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_3/design.csv")

# 0) train, test 데이터 분리
cond1 = df['id'] <= 140
cond2 = df['id'] > 140
train = df[cond1].copy()
test = df[cond2].copy()

# 1) 유의한 변수만 사용한 회귀 분석 => c1, c2, c4
from statsmodels.formula.api import ols
model = ols('design ~ c1+c2+c4', data=train).fit()

# 2) train 데이터에서 design의 예측
train['design_pred'] = model.predict(train)
print(train)

# 3) 피어슨 상관계수 계산
result = train['design'].corr(train['design_pred'])
print(round(result, 3))

      id     design        c1        c2        c3        c4        c5  \
0      1  58.962471  0.374540  0.642032  0.103124  0.168935  0.707239   
1      2  42.006534  0.950714  0.084140  0.902553  0.278590  0.152539   
2      3  55.831980  0.731994  0.161629  0.505252  0.177010  0.576288   
3      4  64.458592  0.598658  0.898554  0.826457  0.088703  0.606715   
4      5  61.342792  0.156019  0.606429  0.320050  0.120636  0.424131   
..   ...        ...       ...       ...       ...       ...       ...   
135  136  58.457931  0.323203  0.040729  0.944766  0.877201  0.272307   
136  137  66.020866  0.518791  0.855461  0.849647  0.392445  0.018391   
137  138  63.945956  0.703019  0.703658  0.247348  0.816599  0.914299   
138  139  54.083051  0.363630  0.474174  0.450544  0.439135  0.117751   
139  140  55.031334  0.971782  0.097834  0.129159  0.376944  0.576516   

     design_pred  
0      60.071154  
1      51.703419  
2      53.004302  
3      61.348118  
4      60.437781  
..       

In [193]:
# 방법2(추가tip)

# 1) 유의한 변수만 사용한 회귀 분석
from statsmodels.formula.api import ols
model = ols("design ~ c1 + c2 + c4", data=train).fit()

# 학습데이터에서 예측값 계산
train['pred_design'] = model.predict(train)
print(train)

# 예측값과 실제 design 값 사이의 피어슨 상관계수 계산
from scipy.stats import pearsonr
c, p = pearsonr(train['design'], train['pred_design'])
print(round(c, 3))

      id     design        c1        c2        c3        c4        c5  \
0      1  58.962471  0.374540  0.642032  0.103124  0.168935  0.707239   
1      2  42.006534  0.950714  0.084140  0.902553  0.278590  0.152539   
2      3  55.831980  0.731994  0.161629  0.505252  0.177010  0.576288   
3      4  64.458592  0.598658  0.898554  0.826457  0.088703  0.606715   
4      5  61.342792  0.156019  0.606429  0.320050  0.120636  0.424131   
..   ...        ...       ...       ...       ...       ...       ...   
135  136  58.457931  0.323203  0.040729  0.944766  0.877201  0.272307   
136  137  66.020866  0.518791  0.855461  0.849647  0.392445  0.018391   
137  138  63.945956  0.703019  0.703658  0.247348  0.816599  0.914299   
138  139  54.083051  0.363630  0.474174  0.450544  0.439135  0.117751   
139  140  55.031334  0.971782  0.097834  0.129159  0.376944  0.576516   

     pred_design  
0      60.071154  
1      51.703419  
2      53.004302  
3      61.348118  
4      60.437781  
..       

### 문제1-3
문제 1-2에서 적합한 회귀모형을 이용하여 test 데이터에서 design의 예측값을 산출한 후, 예측값과 실제 design 값 사이의 RMSE (Root Mean Squared Error)를 반올림하여 소수 셋째 자리까지 계산하시오.

In [203]:
# 1) test데이터에서 design값 계산
test['design_pred'] = model.predict(test)
print(test.head())

# 2) test데이터에 대한 RMSE 계산
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(test['design'], test['design_pred'])
print(round(rmse, 3))

      id     design        c1        c2        c3        c4        c5  \
140  141  66.574168  0.962447  0.491616  0.954051  0.462680  0.274055   
141  142  48.521635  0.251782  0.473472  0.606175  0.301378  0.554178   
142  143  57.754669  0.497249  0.173202  0.228643  0.747609  0.651420   
143  144  52.072094  0.300878  0.433852  0.671701  0.502720  0.829742   
144  145  65.085687  0.284840  0.398505  0.618128  0.232213  0.206421   

     design_pred  
140    57.350734  
141    59.608992  
142    57.831803  
143    60.180171  
144    58.178462  
8.488


## 문제2. 로지스틱 회귀
- 종속 변수 (타겟 변수): Churn (고객 이탈 여부; 1: 이탈, 0: 유지)
- 독립 변수 (설명 변수):
    - MonthlyCharges (월 사용 요금)
    - CustomerTenure (고객 유지 기간)
    - HasPhoneService (전화 서비스 이용 여부)
    - HasTechInsurance (기술 보험 가입 여부)
    
### 문제2-1
주어진 데이터를 사용하여 고객 이탈(Churn)을 예측하는 로지스틱 회귀 모형을 적합한 후, MonthlyCharges 변수의 p-value를 반올림하여 소수 셋째 자리까지 구하시오.

In [210]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_3/retention.csv")
df.head()

,CustomerID,MonthlyCharges,CustomerTenure,HasPhoneService,HasTechInsurance,Churn
0,1,77.450712,47,1,1,1
1,2,67.926035,68,1,1,0
2,3,79.715328,45,0,1,1
3,4,92.845448,2,1,0,1
4,5,66.487699,27,1,1,0


In [214]:
from statsmodels.formula.api import logit

# 1) 로지스틱 회귀분석 수행
formula = 'Churn ~ MonthlyCharges + CustomerTenure + HasPhoneService + HasTechInsurance'
model = logit(formula, data = df).fit()

# 2) 학습된 모델의 회귀 분석 결과 출력
print(model.summary())

# 3) 'MonthlyCharges' 변수의 p-value 확인
print(round(model.pvalues['MonthlyCharges'], 3))

Optimization terminated successfully.
         Current function value: 0.582234
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                   80
Model:                          Logit   Df Residuals:                       75
Method:                           MLE   Df Model:                            4
Date:                Fri, 20 Jun 2025   Pseudo R-squ.:                  0.1585
Time:                        22:54:16   Log-Likelihood:                -46.579
converged:                       True   LL-Null:                       -55.352
Covariance Type:            nonrobust   LLR p-value:                  0.001513
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -4.4731      1.437     -3.114      0.002      -7.289      -1.657
MonthlyChar

### 문제2-2
적합한 로지스틱 회귀모형을 바탕으로, 전화 서비스를 받지 않은 고객이 이탈할 확률 대비 전화 서비스를 받은 고객이 이탈할 확률에 대한 오즈비(Odds Ratio)를 계산하시오. (반올림하여 소수 셋째 자리까지 계산)

In [219]:
# 1) 오즈비 계산
import numpy as np
result = np.exp(model.params['HasPhoneService'])
print(round(result, 3))

0.701


### 문제2-3
적합한 로지스틱 회귀모형을 이용하여 각 고객의 이탈 확률을 예측한 후, 예측된 이탈 확률이 0.3 초과하는 고객의 수를 구하시오.

In [220]:
# 1) 각 고객의 이탈 확률을 예측
pred = model.predict(df)
print(pred)

# 2) 예측한 이탈 확률이 0.3 초과하는 고객의 수
sum(pred > 0.3)

0     0.643397
1     0.733174
2     0.725944
3     0.480867
4     0.306304
        ...   
75    0.516097
76    0.810504
77    0.355781
78    0.556763
79    0.166611
Length: 80, dtype: float64


65

# 기출 8회_작업형3

## 문제1

### 문제1-1
주어진 데이터에서 로지스틱 회귀 분석을 수행해 유의확률(p-value)이 0.05 이상인 유의하지 않은 독립변수의 개수를 구하시오. (종속변수: Churn)

In [68]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/8_3/churn.csv")
df.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,51,1,0,1.0,0,303.7,83,72.5,9.3,3.8
1,1,61,1,0,-0.2,2,194.8,89,66.3,10.9,13.3
2,0,84,1,1,-1.3,2,157.6,109,35.7,10.0,12.5
3,0,65,1,0,1.0,1,223.1,129,64.2,9.1,5.1
4,0,154,1,0,1.0,0,137.0,115,40.0,7.7,4.6


In [66]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/8_3/churn.csv")

from statsmodels.formula.api import logit

# 1) 로지스틱 회귀분석을 위한 포뮬라 생성
formula = 'Churn ~ AccountWeeks + ContractRenewal + DataPlan + DataUsage + CustServCalls + DayMins + DayCalls + MonthlyCharge + OverageFee + RoamMins'

# 2) 로지스틱 회귀 모델 학습
model = logit(formula, data=df).fit()

# 3) 유의하지 않은 독립변수의 개수 구하기
print(model.summary())
print(sum(model.pvalues[1:] >= 0.05))

Optimization terminated successfully.
         Current function value: 0.393603
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      989
Method:                           MLE   Df Model:                           10
Date:                Fri, 20 Jun 2025   Pseudo R-squ.:                 0.02367
Time:                        23:12:18   Log-Likelihood:                -393.60
converged:                       True   LL-Null:                       -403.14
Covariance Type:            nonrobust   LLR p-value:                   0.03924
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.2146      0.913     -2.425      0.015      -4.005      -0.424
AccountWeeks  

### 문제1-2
유의확률(p-value)이 0.05 미만인 유의한 변수만을 사용해 다시 로지스틱 회귀 분석을 수행하시오. 이 회귀식의 유의한 회귀 계수(상수항 포함)의 합계를 구하시오. (반올림하여 소수 셋째 자리까지 계산)

In [71]:
# 1) 유의한 변수 선택
print(model.pvalues[1:] < 0.05)
formula = "Churn ~ DataUsage + DayMins"

# 2) 로지스틱 회귀
model = logit(formula, data=df).fit()

# 3) 유의한 회귀계수 합계
print(model.summary())
print(round(sum(model.params),3))

DataUsage    True
DayMins      True
dtype: bool
Optimization terminated successfully.
         Current function value: 0.397599
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      997
Method:                           MLE   Df Model:                            2
Date:                Fri, 20 Jun 2025   Pseudo R-squ.:                 0.01375
Time:                        23:25:21   Log-Likelihood:                -397.60
converged:                       True   LL-Null:                       -403.14
Covariance Type:            nonrobust   LLR p-value:                  0.003908
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0395      0.303     -3.434      0.001      

### 문제1-3
문제 1-2에서 수행한 로지스틱 회귀식에서 ‘DataUsage’ 변수가 5만큼 증가할 때 오즈비(Odds Ratio)를
구하시오. (반올림하여 소수 셋째 자리까지 계산)

In [ ]:
# 1) 변수의 회귀계수 추출
import numpy as np
coef = model.params['DataUsage']

# 2) 오즈비 계산
round(np.exp(coef * 5),3)

0.428

## 문제2

### 문제 2-1
주어진 데이터를 이용해 종속변수(PIQ)와 독립변수(Brain, Height, Weight)로 다중 선형 회귀 분석을 수행
하시오. 이때 유의확률(p-value)이 가장 작은 변수의 회귀 계수 값을 구하시오. (반올림하여 소수 셋째 자리까지 계산)

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/8_3/piq.csv")
df.head()

,PIQ,Brain,Height,Weight
0,132,85.78,62.5,127
1,96,86.54,68.0,135
2,84,90.49,66.3,134
3,134,79.06,62.0,122
4,86,88.91,70.0,180


In [ ]:
# 1) 다중 선형 회귀 분석
from statsmodels.formula.api import ols
model = ols('PIQ ~ Brain + Height + Weight', data=df).fit()

# 2) 가장 작은 p-value를 가진 변수의 회귀계수 찾기
print(model.summary())

# 정답: 2.3431

                            OLS Regression Results                            
Dep. Variable:                    PIQ   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     9.024
Date:                Sat, 21 Sep 2024   Prob (F-statistic):           8.25e-05
Time:                        06:45:49   Log-Likelihood:                -217.27
No. Observations:                  50   AIC:                             442.5
Df Residuals:                      46   BIC:                             450.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.5114     50.564      1.474      0.1

### 문제 2-2
문제 2-1에 적합한 모델의 결정 계수(R-squared) 값을 구하시오. (반올림하여 소수 둘째 자리까지 계산)

In [ ]:
# 1) 결정계수(R-squared) 값
print(round(model.rsquared, 2))

0.37


### 문제 2-3
뇌 크기(Brain)가 90, 키(Height)가 70, 몸무게(Weight)가 150일 때의 PIQ를 예측하시오. (반올림하여 정수로 계산)

In [ ]:
# 1) 새로운 데이터 생성
new_data = pd.DataFrame({'Brain':[90], 'Height':[70], 'Weight':[150]})

# 2) PIQ 예측
pred = model.predict(new_data)
print(pred)
print(round(pred[0]))

0    106.38302
dtype: float64
106


# 기출 7회_작업형3

## 문제1
주어진 조개 데이터 300개 중 앞에서부터 210개는 train 데이터로 만들고, 나머지 90개는 test데이터로 만든다. 모델을 학습(적합)할 때는 train데이터를 사용하고, 예측할 때는 test데이터를 사용한다. 모델은 로지스틱 회귀를 써서 성별(gender)을 예측하되, 패널티는 부과하지 않는다.
- 종속변수: gender(0:암컷, 1:수컷)

### 문제1-1. weight를 독립변수로 gender를 종속변수로 사용하여 로지스틱 회귀 모형을 만들고, weight 변수가 한 단위 증가할 때 수컷일 오즈비 값은? (반올림하여 소수 넷째자리까지 계산)

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/clam.csv")

# 데이터셋 분할
train = df.iloc[:210]
test = df.iloc[210:]
print(train.shape, test.shape)

(210, 6) (90, 6)


In [ ]:
from statsmodels.formula.api import logit
import numpy as np

# 1) 로지스틱 회귀 모델 생성 및 학습
model = logit("gender ~ weight", data=train).fit()
print(model.summary())

# 2) 오즈비 계산
print(round(np.exp(0.0047),4))
print(round(np.exp(model.params['weight']),4))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      208
Method:                           MLE   Df Model:                            1
Date:                Sat, 21 Sep 2024   Pseudo R-squ.:                0.003431
Time:                        05:54:09   Log-Likelihood:                -144.91
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.3178
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3140      0.276     -1.137      0.256      -0.855       0.227
weight         0.0047      0.

In [ ]:
# [참고] <심화학습> 암컷의 오즈비
female_odds_ratio = round(np.exp(-model.params['weight']), 4)
print(female_odds_ratio)

0.9953


### 문제1-2. gender를 종속변수로 하고 나머지 변수들(age, length, diameter, height, weight)을 독립변수로 사용하는 로지스틱 회귀 모델을 적합시킨 후, 잔차 이탈도(Residual Deviance)를 계산하시오. (반올림하여 소수 둘째자리까지 계산)

In [ ]:
# 방법1
from statsmodels.formula.api import logit

# 1) 로지스틱 회귀 모델 생성 및 학습
model = logit("gender ~ age + length + diameter + height + weight", data=train).fit()
print(model.summary())

# 2) 잔차 이탈도 계산
print(round(-2 * model.llf,2))

Optimization terminated successfully.
         Current function value: 0.683173
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      204
Method:                           MLE   Df Model:                            5
Date:                Sat, 21 Sep 2024   Pseudo R-squ.:                 0.01336
Time:                        05:54:09   Log-Likelihood:                -143.47
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.5662
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6701      0.651      1.030      0.303      -0.605       1.946
age           -0.0482      0.

In [ ]:
# 방법2
from statsmodels.formula.api import glm
import statsmodels.api as sm

# 1) glm 모델 적합 (로지스틱 회귀를 위해 이항 분포 사용)
formula = "gender ~ age + length + diameter + height + weight"
model = glm(formula, data=train, family=sm.families.Binomial()).fit()

# 2) 잔차이탈도 계산
print(model.summary())
print(round(model.deviance,2))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 gender   No. Observations:                  210
Model:                            GLM   Df Residuals:                      204
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -143.47
Date:                Sat, 21 Sep 2024   Deviance:                       286.93
Time:                        05:54:09   Pearson chi2:                     210.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.01833
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6701      0.651      1.030      0.3

### 문제1-3. 독립변수 weight 만 사용해 학습한 로지스틱 회귀모델에서 test데이터의 gender를 예측하고, error rate(오류율)를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
from sklearn.metrics import accuracy_score
from statsmodels.formula.api import logit

# 데이터셋 분할
train = df.iloc[:210]
test = df.iloc[210:]

# 1) 학습, test데이터를 사용해 예측 (0.5 미만: 0, 0.5 이상 1)
model = logit("gender ~ weight", data=train).fit()
target = test.pop("gender")
pred = model.predict(test) > 0.5

# 2) 실제 값과 예측 값을 사용하여 정확도 계산
acc = accuracy_score(target, pred)

# 3) 오류율 계산
print(round(1-acc,3))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
0.478


In [ ]:
# 다른 풀이 (사이킷런)
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

train = df.iloc[:210]
test = df.iloc[210:]

# 1) 학습 데이터와 테스트 데이터 분리
X_train = train[['weight']]
y_train = train['gender']
X_test = test[['weight']]
y_test = test['gender']

# 2) 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 3) 테스트 데이터를 사용해 예측
pred = model.predict(X_test)

# 4) 실제 값과 예측값을 사용하여 정확도 계산
acc = accuracy_score(y_test, pred)

# 5) 오류율 계산
print(round(1 - acc, 3))

0.478


## 문제2

### 문제2-1. ERP와 가장 상관관계 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/system.csv")
df.head(3)

,ERP,Feature1,Feature2,Feature3,CPU
0,135.281047,235.088034,44.462272,-36.532634,20.492369
1,108.003144,36.571914,46.409312,-21.709347,34.169811
2,119.574760,52.239055,66.494894,-30.590820,67.624248


In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/system.csv")

# 1) ERP와 각 변수 사이의 상관계수 계산
df.corr()
# 0.434

,ERP,Feature1,Feature2,Feature3,CPU
ERP,1.000000,0.434442,0.076116,-0.059888,-0.201027
Feature1,0.434442,1.000000,0.076148,0.049980,-0.098571
Feature2,0.076116,0.076148,1.000000,0.126589,0.071456
Feature3,-0.059888,0.049980,0.126589,1.000000,-0.035247
CPU,-0.201027,-0.098571,0.071456,-0.035247,1.000000


### 문제2-2. CPU 컬럼이 100미만인 것만 찾아 ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정해 선형회귀 모델을 만들고 적합한 결정계수를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
from statsmodels.formula.api import ols

# 1) CPU가 100 미만인 데이터 필터링
cond = df['CPU'] < 100
df = df[cond]

# 2) 선형회귀 모델 생성: ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정
model = ols('ERP ~ Feature1 + Feature2 + Feature3 + CPU', data=df).fit()

# 3) 모델 요약 정보 출력
print(model.summary())
print(model.rsquared)
print(model.rsquared_adj)

                            OLS Regression Results                            
Dep. Variable:                    ERP   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     6.937
Date:                Sat, 21 Sep 2024   Prob (F-statistic):           6.07e-05
Time:                        05:54:10   Log-Likelihood:                -429.44
No. Observations:                 100   AIC:                             868.9
Df Residuals:                      95   BIC:                             881.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.4348     15.094      4.932      0.0

### 문제 2-3. 2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
# pvalue
# 0.457
print(model.pvalues)

Intercept    0.000003
Feature1     0.000014
Feature2     0.456708
Feature3     0.298206
CPU          0.067968
dtype: float64


In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/system.csv")
sum(df['CPU'] < 100)
df

,ERP,Feature1,Feature2,Feature3,CPU
0,135.281047,235.088034,44.462272,-36.532634,20.492369
1,108.003144,36.571914,46.409312,-21.709347,34.169811
2,119.574760,52.239055,66.494894,-30.590820,67.624248
3,144.817864,196.195890,59.828956,-33.400891,87.923476
4,137.351160,75.175619,59.601973,-26.668085,54.367805
...,...,...,...,...,...
95,114.131463,105.039508,67.053370,-33.807461,33.747738
96,100.210000,141.114900,51.465875,-25.710380,97.049369
97,135.717410,179.363130,58.744305,-24.294491,13.343943
98,102.538242,217.189747,44.008265,-22.667106,9.680395


# 기출 6회_작업형3

## 문제1
항암약 위약을 투여 받은 환자의 부작용은 감기약 위약을 투여 받은 환자의 부작용 분포와 차이가 있는가?

항암약(위약)을 투여 받은 환자 그룹의 관찰된 부작용

In [ ]:
import pandas as pd
df = pd.DataFrame({ "항암약":[4,4,3,4,1,4,1,4,1,4,4,2,1,4,2,3,2,4,4,4] })
# 1: '아픔', 2: '조금 아픔', 3: '속쓰림', 4: '무증상'

감기약(위약)을 투여받은 환자들 중의 알려진 부작용 발생 비율


In [ ]:
# 1. 아픔: 10%
# 2. 조금 아픔 5%
# 3. 속 쓰림 15%
# 4. 무증상: 70%

-	귀무가설(H0): 감기약(위약)의 부작용과 항암약(위약)의 부작용은 동일하다.
- 	대립가설(H1): 감기약(위약)의 부작용과 항암약(위약)의 부작용은 다르다.

 ### 1-1. 항암약을 투여 받은 환자 중 '무증상'의 비율을 0과 1사이로 구하시오.

In [ ]:
cond = df['항암약'] == 4
sum(cond) / len(df['항암약'])

0.55

In [ ]:
df['항암약'].value_counts(normalize=True)

,proportion
항암약,
4,0.55
1,0.20
2,0.15
3,0.10


### 1-2. 감기약의 예상 부작용 비율과 항암약의 부작용 관찰값이 통계적으로 유의미하게 차이가 있는지 확인하려 한다. 카이 제곱 검정을 사용하여 검정 통계량을 구하시오.

In [ ]:
ob = df['항암약'].value_counts().sort_index().to_list()
ex = [0.1 * 20, 0.05 * 20, 0.15 * 20, 0.7 * 20]

from scipy import stats
stats.chisquare(ob, ex)

Power_divergenceResult(statistic=6.976190476190476, pvalue=0.07266054733847573)

In [ ]:
6.976190476190476

6.976190476190476

### 1-3. 위의 p-값을 구하시오

In [ ]:
0.07266054733847573

0.07266054733847573

## 문제2. 다중 선형 회귀 문제

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/6_3/data6-3-2.csv")
df.head(2)

,solar,wind,o3,temperature
0,89.14,6.28,33.52,23.0
1,109.97,1.04,27.01,20.7


1. 다중 선형 회귀 모델을 구축하고, 독립변수 o3의 회귀계수를 구하시오.
- 독립변수: solar(태양 에너지), wind(바람의 세기), o3(오존 농도)
- 종속변수: temperature(온도)


In [ ]:
from statsmodels.formula.api import ols
formula = "temperature ~ solar + wind + o3"
model = ols(formula, data=df).fit()
print(model.summary())
# 0.0749
model.params['o3']

                            OLS Regression Results                            
Dep. Variable:            temperature   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.464
Date:                Sat, 21 Sep 2024   Prob (F-statistic):              0.229
Time:                        05:45:45   Log-Likelihood:                -195.45
No. Observations:                 100   AIC:                             398.9
Df Residuals:                      96   BIC:                             409.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.0507      1.994      9.555      0.0

0.0749385437813658

2. 데이터에서 'solar'와 'o3' 값을 고정한 상태에서, 'wind'의 세기가 증가함에 따라 'temperature'가 감소하는지를 검증하기 위해 다중 선형 회귀 분석을 수행하고, 이 때'wind'의 회귀 계수에 대한 p-value 값을 구하시오. (유의수준: 0.05)

In [ ]:
0.78
model.pvalues['wind']

0.7797177202071661

3. solar:100, wind:5, o3:30일 때 예측값과 그에 대한 95% 신뢰구간을 구하시오.

In [ ]:
new_data = pd.DataFrame({
    'solar':[100],
    'wind':[5],
    'o3':[30]
})
pred = model.get_prediction(new_data)
print(pred.summary_frame(alpha=0.05))

       mean   mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  21.56163  0.175263      21.213737      21.909524     18.082985   

   obs_ci_upper  
0     25.040276  


In [ ]:
# 예측값 21.56163
# 예측값에 대한 95% 신뢰구간 21.213737      21.909524